# Coleta de Valores de Combustíveis por Região

**Projeto:** Top One Model - Sistema de Modelagem de Risco de Crédito

**Objetivo:**
- Fazer coleta automática dos dados de preços de combustíveis da ANP
- Salvar dados brutos (raw_data) para posterior análise
- Realizar feature engineering nos dados coletados

**Fonte dos Dados:**
https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/levantamento-de-precos-de-combustiveis-ultimas-semanas-pesquisadas

**Estrutura de Dados:**
```
data/external_data/macro_specified_data/fuel_prices/
├── raw_data/           # Dados brutos da ANP
└── processed_data/     # Dados processados (feature engineering)
```

## 1. Configuração Inicial e Importações

In [1]:
# Importações essenciais
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from pathlib import Path
from io import BytesIO
from datetime import datetime
import re
from urllib.parse import urljoin
import warnings
warnings.filterwarnings('ignore')

print("🚀 COLETA DE VALORES DE COMBUSTÍVEIS POR REGIÃO")
print("=" * 60)
print("📅 Data de execução:", datetime.now().strftime('%d/%m/%Y %H:%M:%S'))
print("🌐 Fonte: ANP - Agência Nacional do Petróleo")
print("✅ Bibliotecas importadas com sucesso!")

🚀 COLETA DE VALORES DE COMBUSTÍVEIS POR REGIÃO
📅 Data de execução: 04/08/2025 17:02:54
🌐 Fonte: ANP - Agência Nacional do Petróleo
✅ Bibliotecas importadas com sucesso!


## 2. Configuração de Pastas e Estrutura de Dados

In [2]:
# Configuração das pastas
project_root = Path('/home/usuario/Documentos/top_one_model_01')
fuel_prices_root = project_root / 'data' / 'external_data' / 'macro_specified_data' / 'fuel_prices'

# Criar estrutura de pastas
raw_data_path = fuel_prices_root / 'raw_data'
processed_data_path = fuel_prices_root / 'processed_data'

# Criar todas as pastas necessárias
raw_data_path.mkdir(parents=True, exist_ok=True)
processed_data_path.mkdir(parents=True, exist_ok=True)

print("📁 ESTRUTURA DE PASTAS CRIADA:")
print(f"   📂 Projeto: {project_root}")
print(f"   📂 Raw Data: {raw_data_path}")
print(f"   📂 Processed Data: {processed_data_path}")
print("✅ Estrutura configurada com sucesso!")

📁 ESTRUTURA DE PASTAS CRIADA:
   📂 Projeto: /home/usuario/Documentos/top_one_model_01
   📂 Raw Data: /home/usuario/Documentos/top_one_model_01/data/external_data/macro_specified_data/fuel_prices/raw_data
   📂 Processed Data: /home/usuario/Documentos/top_one_model_01/data/external_data/macro_specified_data/fuel_prices/processed_data
✅ Estrutura configurada com sucesso!


## 3. Função de Webscraping da Página ANP

In [3]:
def extrair_links_combustiveis_anp():
    """
    Extrai todos os links de arquivos Excel de combustíveis da página oficial da ANP
    """
    
    url_anp = "https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/levantamento-de-precos-de-combustiveis-ultimas-semanas-pesquisadas"
    
    print("🔍 Acessando página oficial da ANP...")
    print(f"🌐 URL: {url_anp}")
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'pt-BR,pt;q=0.9,en;q=0.8',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }
        
        response = requests.get(url_anp, headers=headers, timeout=30)
        
        if response.status_code != 200:
            print(f"❌ Erro ao acessar página: Status {response.status_code}")
            return []
        
        print(f"✅ Página acessada com sucesso (Status: {response.status_code})")
        
        # Parse do HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontrar todos os links para arquivos Excel
        links_excel = []
        
        for link in soup.find_all('a', href=True):
            href = link['href']
            
            # Verificar se é um link para arquivo Excel da ANP
            if (href.endswith('.xlsx') and 
                'arquivos-lpc' in href and
                ('resumo' in href.lower() or 'revendas' in href.lower())):
                
                # Garantir URL absoluta
                if href.startswith('http'):
                    url_completa = href
                else:
                    url_completa = urljoin(url_anp, href)
                
                # Extrair informações do link
                texto_link = link.get_text(strip=True)
                
                # Determinar tipo baseado no URL e texto
                if 'resumo' in href.lower() or 'médios semanais' in texto_link.lower():
                    tipo = 'resumo'
                elif 'revendas' in href.lower() or 'posto revendedor' in texto_link.lower():
                    tipo = 'revendas'
                else:
                    tipo = 'desconhecido'
                
                # Extrair período do nome do arquivo
                periodo_match = re.search(r'(\d{4}-\d{2}-\d{2})_(\d{4}-\d{2}-\d{2})', href)
                if periodo_match:
                    periodo = f"{periodo_match.group(1)}_{periodo_match.group(2)}"
                    ano = periodo_match.group(1)[:4]
                    
                    links_excel.append({
                        'url': url_completa,
                        'tipo': tipo,
                        'periodo': periodo,
                        'ano': int(ano),
                        'texto_link': texto_link,
                        'nome_arquivo': href.split('/')[-1]
                    })
        
        print(f"📋 Links encontrados: {len(links_excel)}")
        
        # Organizar por tipo e ano
        por_tipo = {}
        for link in links_excel:
            tipo = link['tipo']
            if tipo not in por_tipo:
                por_tipo[tipo] = []
            por_tipo[tipo].append(link)
        
        print(f"📊 DISTRIBUIÇÃO POR TIPO:")
        for tipo, links in por_tipo.items():
            anos = set(link['ano'] for link in links)
            print(f"   • {tipo}: {len(links)} arquivos ({len(anos)} anos)")
        
        return sorted(links_excel, key=lambda x: (x['ano'], x['periodo']), reverse=True)
        
    except Exception as e:
        print(f"❌ Erro no webscraping: {e}")
        return []

print("✅ Função de extração de links criada!")

✅ Função de extração de links criada!


## 4. Função de Download dos Arquivos

In [4]:
def baixar_arquivo_combustivel(link_info, pasta_destino):
    """
    Baixa um arquivo específico de combustível da ANP
    """
    
    url = link_info['url']
    nome_arquivo = f"{link_info['tipo']}_{link_info['periodo']}.xlsx"
    arquivo_path = pasta_destino / nome_arquivo
    
    print(f"📥 Baixando: {nome_arquivo}")
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36',
            'Accept': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet,application/vnd.ms-excel,*/*',
            'Referer': 'https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/'
        }
        
        response = requests.get(url, headers=headers, timeout=30, allow_redirects=True)
        
        if response.status_code == 200 and len(response.content) > 1000:
            # Salvar arquivo bruto
            with open(arquivo_path, 'wb') as f:
                f.write(response.content)
            
            # Tentar ler como Excel para validar
            try:
                df = pd.read_excel(BytesIO(response.content), engine='openpyxl')
                
                tamanho_mb = len(response.content) / (1024 * 1024)
                print(f"   ✅ Sucesso: {len(df):,} registros, {len(df.columns)} colunas ({tamanho_mb:.1f} MB)")
                
                return {
                    'sucesso': True,
                    'arquivo': arquivo_path,
                    'registros': len(df),
                    'colunas': len(df.columns),
                    'tamanho_mb': tamanho_mb,
                    'dados': df
                }
                
            except Exception as e_excel:
                print(f"   ⚠️ Arquivo salvo mas com erro na leitura: {e_excel}")
                return {
                    'sucesso': False,
                    'arquivo': arquivo_path,
                    'erro': f"Erro na leitura Excel: {e_excel}"
                }
        else:
            print(f"   ❌ Arquivo inválido: Status {response.status_code}, Tamanho {len(response.content)} bytes")
            return {
                'sucesso': False,
                'erro': f"Status {response.status_code}, tamanho inválido"
            }
            
    except Exception as e:
        print(f"   ❌ Erro no download: {e}")
        return {
            'sucesso': False,
            'erro': str(e)
        }

print("✅ Função de download criada!")

✅ Função de download criada!


## 5. Execução da Coleta de Raw Data

In [5]:
def coletar_todos_raw_data():
    """
    Executa a coleta completa dos dados brutos de combustíveis
    """
    
    print("🚀 INICIANDO COLETA DE RAW DATA - COMBUSTÍVEIS ANP")
    print("=" * 70)
    
    # 1. Extrair todos os links
    print("\n🔍 FASE 1: EXTRAINDO LINKS DA PÁGINA OFICIAL...")
    links = extrair_links_combustiveis_anp()
    
    if not links:
        print("❌ Nenhum link encontrado! Abortando coleta.")
        return None
    
    print(f"✅ {len(links)} links extraídos com sucesso")
    
    # 2. Executar downloads
    print(f"\n📥 FASE 2: BAIXANDO ARQUIVOS...")
    print("=" * 50)
    
    resultados = []
    estatisticas = {
        'total': len(links),
        'sucessos': 0,
        'falhas': 0,
        'tamanho_total': 0,
        'registros_total': 0
    }
    
    for i, link in enumerate(links, 1):
        print(f"\n📄 [{i}/{len(links)}] {link['periodo']} ({link['tipo']})")
        
        # Baixar arquivo
        resultado = baixar_arquivo_combustivel(link, raw_data_path)
        
        # Adicionar informações do link ao resultado
        resultado['link_info'] = link
        resultados.append(resultado)
        
        if resultado['sucesso']:
            estatisticas['sucessos'] += 1
            if 'tamanho_mb' in resultado:
                estatisticas['tamanho_total'] += resultado['tamanho_mb']
            if 'registros' in resultado:
                estatisticas['registros_total'] += resultado['registros']
        else:
            estatisticas['falhas'] += 1
        
        # Pausa entre downloads para não sobrecarregar o servidor
        time.sleep(1)
        
        # Mostrar progresso a cada 5 arquivos
        if i % 5 == 0:
            taxa_sucesso = (estatisticas['sucessos'] / i) * 100
            print(f"\n   📊 Progresso: {i}/{len(links)} ({taxa_sucesso:.1f}% sucesso)")
    
    # 3. Relatório final
    print(f"\n🏁 COLETA DE RAW DATA CONCLUÍDA!")
    print("=" * 50)
    print(f"📊 ESTATÍSTICAS FINAIS:")
    print(f"   • Total processado: {estatisticas['total']}")
    print(f"   • ✅ Sucessos: {estatisticas['sucessos']}")
    print(f"   • ❌ Falhas: {estatisticas['falhas']}")
    print(f"   • 📈 Taxa de sucesso: {(estatisticas['sucessos']/estatisticas['total']*100):.1f}%")
    print(f"   • 💾 Tamanho total: {estatisticas['tamanho_total']:.1f} MB")
    print(f"   • 📋 Registros totais: {estatisticas['registros_total']:,}")
    
    print(f"\n📂 DADOS SALVOS EM: {raw_data_path}")
    
    # Listar arquivos salvos
    arquivos_salvos = list(raw_data_path.glob('*.xlsx'))
    print(f"\n📁 ARQUIVOS SALVOS ({len(arquivos_salvos)}):")
    for arquivo in sorted(arquivos_salvos):
        tamanho = arquivo.stat().st_size / (1024 * 1024)
        print(f"   • {arquivo.name} ({tamanho:.1f} MB)")
    
    return {
        'resultados': resultados,
        'estatisticas': estatisticas,
        'pasta_raw_data': raw_data_path,
        'arquivos_salvos': arquivos_salvos
    }

print("✅ Função de coleta completa criada!")

✅ Função de coleta completa criada!


## 6. Executar Coleta dos Raw Data

In [6]:
# Executar a coleta completa
print("🎯 INICIANDO PROCESSO DE COLETA DE RAW DATA")
print("💡 Isso pode levar alguns minutos dependendo da quantidade de arquivos...")
print()

resultado_coleta = coletar_todos_raw_data()

if resultado_coleta:
    print(f"\n🎉 COLETA CONCLUÍDA COM SUCESSO!")
    print(f"📊 {resultado_coleta['estatisticas']['sucessos']} arquivos coletados")
    print(f"📂 Dados salvos em: {resultado_coleta['pasta_raw_data']}")
    
    # Disponibilizar globalmente para próximas células
    globals()['dados_raw_combustivel'] = resultado_coleta
    
else:
    print("❌ Falha na coleta dos dados")

🎯 INICIANDO PROCESSO DE COLETA DE RAW DATA
💡 Isso pode levar alguns minutos dependendo da quantidade de arquivos...

🚀 INICIANDO COLETA DE RAW DATA - COMBUSTÍVEIS ANP

🔍 FASE 1: EXTRAINDO LINKS DA PÁGINA OFICIAL...
🔍 Acessando página oficial da ANP...
🌐 URL: https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/levantamento-de-precos-de-combustiveis-ultimas-semanas-pesquisadas
✅ Página acessada com sucesso (Status: 200)
📋 Links encontrados: 279
📊 DISTRIBUIÇÃO POR TIPO:
   • resumo: 140 arquivos (4 anos)
   • revendas: 139 arquivos (4 anos)
✅ 279 links extraídos com sucesso

📥 FASE 2: BAIXANDO ARQUIVOS...

📄 [1/279] 2025-07-27_2025-08-02 (resumo)
📥 Baixando: resumo_2025-07-27_2025-08-02.xlsx
✅ Página acessada com sucesso (Status: 200)
📋 Links encontrados: 279
📊 DISTRIBUIÇÃO POR TIPO:
   • resumo: 140 arquivos (4 anos)
   • revendas: 139 arquivos (4 anos)
✅ 279 links extraídos com sucesso

📥 FASE 2: BAIXANDO ARQUIVOS...

📄 [1/279] 2025-07-27_2025-08-02 (resumo)
📥 B

## 7. Análise Inicial dos Raw Data Coletados

In [7]:
# Análise dos arquivos coletados
if 'dados_raw_combustivel' in globals():
    print("🔍 ANÁLISE INICIAL DOS RAW DATA COLETADOS")
    print("=" * 60)
    
    dados = dados_raw_combustivel
    
    # Verificar arquivos na pasta
    arquivos = list(raw_data_path.glob('*.xlsx'))
    print(f"\n📁 ARQUIVOS ENCONTRADOS: {len(arquivos)}")
    
    # Analisar alguns arquivos como exemplo
    print(f"\n🔍 ANÁLISE DE ESTRUTURA (primeiros 3 arquivos):")
    
    for i, arquivo in enumerate(sorted(arquivos)[:3]):
        print(f"\n📄 {i+1}. {arquivo.name}")
        
        try:
            df = pd.read_excel(arquivo, engine='openpyxl')
            
            print(f"   📊 Shape: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
            print(f"   📋 Colunas: {list(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
            
            # Verificar se há dados geográficos
            colunas_geo = [col for col in df.columns if any(termo in col.lower() for termo in ['estado', 'uf', 'município', 'regiao'])]
            if colunas_geo:
                print(f"   🗺️ Colunas geográficas: {colunas_geo[:3]}")
            
            # Verificar se há dados de preços
            colunas_preco = [col for col in df.columns if any(termo in col.lower() for termo in ['preço', 'preco', 'valor'])]
            if colunas_preco:
                print(f"   💰 Colunas de preço: {colunas_preco[:3]}")
            
        except Exception as e:
            print(f"   ❌ Erro ao ler arquivo: {e}")
    
    print(f"\n✅ ANÁLISE INICIAL CONCLUÍDA!")
    print(f"🎯 PRÓXIMO PASSO: Feature Engineering nos dados coletados")
    
else:
    print("⚠️ Execute primeiro a célula de coleta dos dados")

🔍 ANÁLISE INICIAL DOS RAW DATA COLETADOS

📁 ARQUIVOS ENCONTRADOS: 279

🔍 ANÁLISE DE ESTRUTURA (primeiros 3 arquivos):

📄 1. resumo_2022-08-21_2022-08-27.xlsx
   ❌ Erro ao ler arquivo: ExternalReference.__init__() missing 1 required positional argument: 'id'

📄 2. resumo_2022-08-28_2022-09-03.xlsx
   ❌ Erro ao ler arquivo: ExternalReference.__init__() missing 1 required positional argument: 'id'

📄 3. resumo_2022-09-04_2022-09-10.xlsx
   📊 Shape: 181 linhas x 12 colunas
   📋 Colunas: ['AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']...

✅ ANÁLISE INICIAL CONCLUÍDA!
🎯 PRÓXIMO PASSO: Feature Engineering nos dados coletados


## 9. Preparação para Feature Engineering

## 8. Organização dos Raw Data em Pastas Separadas

In [9]:
def organizar_raw_data_por_tipo():
    """
    Organiza os arquivos de raw data em pastas separadas por tipo (resumo e revendas)
    """
    
    print("📁 ORGANIZANDO RAW DATA EM PASTAS SEPARADAS")
    print("=" * 60)
    
    # Criar estrutura de subpastas
    pasta_resumo = raw_data_path / 'resumo'
    pasta_revendas = raw_data_path / 'revendas'
    
    # Criar as pastas se não existirem
    pasta_resumo.mkdir(exist_ok=True)
    pasta_revendas.mkdir(exist_ok=True)
    
    print(f"📂 Criadas pastas:")
    print(f"   • {pasta_resumo}")
    print(f"   • {pasta_revendas}")
    
    # Listar todos os arquivos na pasta raw_data principal
    arquivos_raw = list(raw_data_path.glob('*.xlsx'))
    
    if not arquivos_raw:
        print("⚠️ Nenhum arquivo encontrado na pasta raw_data")
        return
    
    print(f"\n📋 Arquivos encontrados: {len(arquivos_raw)}")
    
    # Contadores para relatório
    movidos_resumo = 0
    movidos_revendas = 0
    nao_identificados = 0
    
    for arquivo in arquivos_raw:
        nome_arquivo = arquivo.name.lower()
        
        # Determinar tipo baseado no nome do arquivo
        if 'resumo' in nome_arquivo:
            # Mover para pasta resumo
            destino = pasta_resumo / arquivo.name
            arquivo.rename(destino)
            movidos_resumo += 1
            print(f"   📄 ➜ resumo/: {arquivo.name}")
            
        elif 'revendas' in nome_arquivo or 'revenda' in nome_arquivo:
            # Mover para pasta revendas
            destino = pasta_revendas / arquivo.name
            arquivo.rename(destino)
            movidos_revendas += 1
            print(f"   📄 ➜ revendas/: {arquivo.name}")
            
        else:
            # Arquivos não identificados permanecem na pasta principal
            nao_identificados += 1
            print(f"   📄 ➜ (mantido): {arquivo.name}")
    
    print(f"\n📊 RELATÓRIO DE ORGANIZAÇÃO:")
    print(f"   • 📋 Resumos: {movidos_resumo} arquivos")
    print(f"   • 🏪 Revendas: {movidos_revendas} arquivos")
    print(f"   • ❓ Não identificados: {nao_identificados} arquivos")
    
    # Verificar estrutura final
    print(f"\n📁 ESTRUTURA FINAL:")
    print(f"   📂 raw_data/")
    
    # Contar arquivos em cada subpasta
    resumos_final = list(pasta_resumo.glob('*.xlsx'))
    revendas_final = list(pasta_revendas.glob('*.xlsx'))
    principais_final = list(raw_data_path.glob('*.xlsx'))
    
    print(f"   ├── 📁 resumo/ ({len(resumos_final)} arquivos)")
    for arquivo in sorted(resumos_final)[:3]:  # Mostrar apenas os primeiros 3
        tamanho = arquivo.stat().st_size / (1024 * 1024)
        print(f"   │   └── {arquivo.name} ({tamanho:.1f} MB)")
    if len(resumos_final) > 3:
        print(f"   │   └── ... e mais {len(resumos_final) - 3} arquivos")
    
    print(f"   ├── 📁 revendas/ ({len(revendas_final)} arquivos)")
    for arquivo in sorted(revendas_final)[:3]:  # Mostrar apenas os primeiros 3
        tamanho = arquivo.stat().st_size / (1024 * 1024)
        print(f"   │   └── {arquivo.name} ({tamanho:.1f} MB)")
    if len(revendas_final) > 3:
        print(f"   │   └── ... e mais {len(revendas_final) - 3} arquivos")
    
    if principais_final:
        print(f"   └── 📄 Arquivos não categorizados: {len(principais_final)}")
        for arquivo in principais_final:
            tamanho = arquivo.stat().st_size / (1024 * 1024)
            print(f"       └── {arquivo.name} ({tamanho:.1f} MB)")
    
    print(f"\n✅ ORGANIZAÇÃO CONCLUÍDA!")
    
    return {
        'pasta_resumo': pasta_resumo,
        'pasta_revendas': pasta_revendas,
        'resumos': len(resumos_final),
        'revendas': len(revendas_final),
        'nao_identificados': len(principais_final)
    }

print("✅ Função de organização criada!")

✅ Função de organização criada!


In [10]:
# Executar organização dos dados por tipo
print("🎯 ORGANIZANDO DADOS COLETADOS POR TIPO")
print("💡 Separando resumos e revendas em pastas específicas...")
print()

resultado_organizacao = organizar_raw_data_por_tipo()

if resultado_organizacao:
    print(f"\n🎉 ORGANIZAÇÃO CONCLUÍDA!")
    print(f"📁 Pasta resumo: {resultado_organizacao['resumos']} arquivos")
    print(f"📁 Pasta revendas: {resultado_organizacao['revendas']} arquivos")
    
    # Atualizar estrutura global
    globals()['estrutura_organizada'] = resultado_organizacao
    
    # Atualizar caminhos para as próximas etapas
    globals()['pasta_resumo'] = resultado_organizacao['pasta_resumo']
    globals()['pasta_revendas'] = resultado_organizacao['pasta_revendas']
    
    print(f"\n✅ Estrutura organizada e pronta para feature engineering!")
    
else:
    print("❌ Falha na organização dos dados")

🎯 ORGANIZANDO DADOS COLETADOS POR TIPO
💡 Separando resumos e revendas em pastas específicas...

📁 ORGANIZANDO RAW DATA EM PASTAS SEPARADAS
📂 Criadas pastas:
   • /home/usuario/Documentos/top_one_model_01/data/external_data/macro_specified_data/fuel_prices/raw_data/resumo
   • /home/usuario/Documentos/top_one_model_01/data/external_data/macro_specified_data/fuel_prices/raw_data/revendas

📋 Arquivos encontrados: 279
   📄 ➜ revendas/: revendas_2024-02-11_2024-02-17.xlsx
   📄 ➜ resumo/: resumo_2025-01-26_2025-02-01.xlsx
   📄 ➜ revendas/: revendas_2023-05-07_2023-05-13.xlsx
   📄 ➜ revendas/: revendas_2025-03-30_2025-04-05.xlsx
   📄 ➜ revendas/: revendas_2023-12-10_2023-12-16.xlsx
   📄 ➜ revendas/: revendas_2024-01-28_2024-02-03.xlsx
   📄 ➜ revendas/: revendas_2023-06-11_2023-06-17.xlsx
   📄 ➜ revendas/: revendas_2023-06-18_2023-06-24.xlsx
   📄 ➜ revendas/: revendas_2025-06-01_2025-06-07.xlsx
   📄 ➜ resumo/: resumo_2023-11-05_2023-11-11.xlsx
   📄 ➜ resumo/: resumo_2025-06-29_2025-07-05.xlsx


In [8]:
print("🔄 PREPARAÇÃO PARA FEATURE ENGINEERING")
print("=" * 50)

# Verificar se temos dados coletados
arquivos_raw = list(raw_data_path.glob('*.xlsx'))

if arquivos_raw:
    print(f"✅ {len(arquivos_raw)} arquivos de raw data disponíveis")
    print(f"📂 Localização: {raw_data_path}")
    
    print(f"\n📋 RESUMO DOS ARQUIVOS:")
    tamanho_total = 0
    for arquivo in sorted(arquivos_raw):
        tamanho = arquivo.stat().st_size / (1024 * 1024)
        tamanho_total += tamanho
        tipo = 'resumo' if 'resumo' in arquivo.name else 'revendas' if 'revendas' in arquivo.name else 'outro'
        print(f"   • {arquivo.name} ({tamanho:.1f} MB) - {tipo}")
    
    print(f"\n💾 Tamanho total: {tamanho_total:.1f} MB")
    
    print(f"\n🎯 PRÓXIMOS PASSOS PARA FEATURE ENGINEERING:")
    print(f"   1. ✅ Coleta de raw data - CONCLUÍDA")
    print(f"   2. 🔄 Análise detalhada da estrutura dos dados")
    print(f"   3. 🔄 Limpeza e padronização")
    print(f"   4. 🔄 Criação de features regionais")
    print(f"   5. 🔄 Agregações por estado/região")
    print(f"   6. 🔄 Salvamento dos dados processados")
    
    print(f"\n🚀 PRONTO PARA INICIAR FEATURE ENGINEERING!")
    
else:
    print("❌ Nenhum arquivo de raw data encontrado")
    print("💡 Execute primeiro as células de coleta de dados")

🔄 PREPARAÇÃO PARA FEATURE ENGINEERING
✅ 279 arquivos de raw data disponíveis
📂 Localização: /home/usuario/Documentos/top_one_model_01/data/external_data/macro_specified_data/fuel_prices/raw_data

📋 RESUMO DOS ARQUIVOS:
   • resumo_2022-08-21_2022-08-27.xlsx (0.3 MB) - resumo
   • resumo_2022-08-28_2022-09-03.xlsx (0.3 MB) - resumo
   • resumo_2022-09-04_2022-09-10.xlsx (0.3 MB) - resumo
   • resumo_2022-09-11_2022-09-17.xlsx (0.3 MB) - resumo
   • resumo_2022-09-18_2022-09-24.xlsx (0.0 MB) - resumo
   • resumo_2022-09-25_2022-10-01.xlsx (0.1 MB) - resumo
   • resumo_2022-10-30_2022-11-05.xlsx (0.3 MB) - resumo
   • resumo_2022-11-06_2022-11-12.xlsx (0.1 MB) - resumo
   • resumo_2022-11-20_2022-11-26.xlsx (0.3 MB) - resumo
   • resumo_2022-12-04_2022-12-10.xlsx (0.3 MB) - resumo
   • resumo_2022-12-11_2022-12-17.xlsx (0.3 MB) - resumo
   • resumo_2022-12-18_2022-12-24.xlsx (0.3 MB) - resumo
   • resumo_2023-01-01_2023-01-07.xlsx (0.3 MB) - resumo
   • resumo_2023-01-08_2023-01-14.xlsx (

---

## ✅ Status da Coleta de Raw Data

**PRIMEIRA FASE CONCLUÍDA:** Coleta automática dos dados brutos da ANP

**Estrutura criada:**
```
data/external_data/macro_specified_data/fuel_prices/
├── raw_data/           # ✅ Dados brutos coletados da ANP
└── processed_data/     # 🔄 Para próxima fase (feature engineering)
```

**Próxima etapa:** Feature Engineering nos dados coletados

---

*Este notebook faz parte do projeto Top One Model - Sistema de Modelagem de Risco de Crédito*